Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

- кошки (животные)
- UNIX-утилита cat для вывода содержимого файлов
- версии операционной системы OS X, названные в честь семейства кошачьих

Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.

Разумеется, использованный нами метод крайне простой. Например, он не учитывает формы слов (так, cat и cats он считает разными словами, хотя по сути они означают одно и то же), не удаляет из текстов артикли и прочие ненужные слова.

In [18]:
import re
import numpy as np
from scipy import spatial

In [19]:
def two_dim_to_one_dim(list1): 
    '''Makes a one-dimensional array from a two-dimensional one'''
    list2 = []
    for arr in list1:
        for elem in arr:
            list2.append(elem)
    return list2

In [20]:
def two_the_most_similar(arr):
    '''Returns an array which contains the two least elements of another array. 
    In our case these elements are the cosine distances'''
    min1 = min2 = arr[0]
    for i in range(len(arr)-1):
        if arr[i+1] < min1:
            min1 = arr[i+1]
    for j in range(len(arr)-2):
            if min1 < arr[j+1] < min2:
                min2 = arr[j+1]
    result = [min1, min2]
    return result

In [21]:
with open('sentences.txt') as file_obj:

    sentences = []

    for line in file_obj:
        words = re.split('[^a-z]', line.lower())
        sentence = list(filter(lambda x: x != '',  words))
        sentences.append(sentence)
    
    words_list = list(set(two_dim_to_one_dim(sentences)))
    words_dict = {word:index for index, word in enumerate(words_list)}

    occurences_matrix = np.zeros((len(sentences), len(words_list)))
    # an element with index (i, j) equals 
    # to the number of occurrences of the j-th word in the i-th sentence

    for number, sentence in enumerate(sentences):
        for word in sentence:
            occurences_matrix[number, words_dict[word]] = sentence.count(word)

    cosine_distances = []

    for k in range(1, len(sentences)):
        cosine = spatial.distance.cosine(occurences_matrix[0:1], occurences_matrix[k:k+1])
        cosine_distances.append(cosine)

    cosine_distances_enumerated = {cosine:index+1 for index, cosine in enumerate(cosine_distances)}

    for elem in two_the_most_similar(cosine_distances):
        print(elem, cosine_distances_enumerated[elem])

0.7327387580875756 6
0.7770887149698589 4
